<a id="1"></a>
<h1 style='background:#92b0ac; border:0; color:black'><center>INTRODUCTION</center></h1>

- Data contain different variable that are sub classified further:
- 1.Demographics of the Employee 
- 2.Tenure Information 
- 3.Historical data regarding performance
- Using these data we will going to find out whether employee will stay in the company and also will see another pattern with respect to time period of person stay in the company. There are other aspects also see in the notebook you can check in visualization part.
- Using this data HR department will find out easily what type of candidate they need to take for particular job role.


<a id="2"></a>
<h1 style='background:#92b0ac; border:0; color:black'><center>LIBRARIES</center></h1>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas import DataFrame
import seaborn as sns

#conda install -c glemaitre imbalanced-learn
from imblearn.under_sampling import NearMiss


# Imputing missing values and scaling values
from sklearn.preprocessing import  MinMaxScaler
from sklearn.impute import SimpleImputer 


import os
for dirname, _, filenames in os.walk('dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


        
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes = True)
%matplotlib inline     


# Machine Learning Helper Functions
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model, decomposition, datasets
from sklearn.pipeline import Pipeline
from pprint import pprint

# Machine Learning Models
from sklearn.svm import SVC
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RandomizedSearchCV



<a id="3"></a>
<h1 style='background:#92b0ac; border:0; color:black'><center>IMPORT_DATA</center></h1>

# 2.Import Data

In [ ]:
#Import dataset
train = pd.read_csv('dataset/train_data.csv')
train.head()

<a id="4"></a>
<h1 style='background:#92b0ac; border:0; color:black'><center>DATA_PREPROCESSING</center></h1>

# 3. Data Preprocessing

In [ ]:
train.isnull().sum()

In [ ]:
train['Target']=np.where(train['LastWorkingDate'].isnull(),0,1)
train.head()

In [ ]:
employees_groupby = train.groupby(['Emp_ID'])['Emp_ID','Age','Gender','City','Education_Level','Salary','Joining Designation','Designation',
                       'Quarterly Rating','Target'].tail(1)
employees_groupby

In [ ]:
employees_groupby.reset_index(inplace = True,drop = True)
employees_groupby

In [ ]:
total_business_groupby = train.groupby('Emp_ID').agg({'Total Business Value':'sum'})
total_business_groupby

In [ ]:
total_business_groupby.reset_index(drop = True,inplace = True)

In [ ]:
final = pd.concat([total_business_groupby,employees_groupby],axis = 1,join = 'inner')
final

corr = final.corr()

plt.figure(figsize = (20,8))
sns.heatmap(corr,annot = True,cmap = 'rocket')

<a id="5"></a>
<h1 style='background:#92b0ac; border:0; color:black'><center>DATA_VISUALIZATION</center></h1>


# 4.Data Visualization

In [ ]:
catplot=sns.catplot(x='Gender',y='Total Business Value',hue = 'Education_Level',data = final,col = 'Education_Level',col_wrap = 2)
catplot.set(xlabel='Gender',ylabel='Total Business Value')

In [ ]:
sns.catplot('Education_Level','Salary',data = final,hue = 'Gender',col = 'City',col_wrap = 2)

In [ ]:
plt.subplots(figsize=(20,10))

plt.subplot(231)
count_plot=sns.countplot(x='Gender',data = final)
count_plot.set(xlabel='Gender',ylabel='')

plt.subplot(232)
sns.countplot(x='Education_Level',data = final)
count_plot.set(xlabel='Education_Level',ylabel='')

plt.subplot(233)
sns.countplot(x='Joining Designation',data = final)
count_plot.set(xlabel='Joining Designation',ylabel='')

plt.subplot(234)
sns.countplot(x='Designation',data = final)
count_plot.set(xlabel='Designation',ylabel='')


In [ ]:
plt.figure(figsize = (20,8))
count_plot=sns.countplot(x='City',data = final,palette= 'rocket')
count_plot.set(xlabel='City',ylabel='')

In [ ]:
count_plot=sns.countplot(x='Target',data = final)
count_plot.set(xlabel='Target',ylabel='')

In [ ]:
final['Quarterly Rating'].unique()

In [ ]:
final ['Gender'] = pd.get_dummies(final['Gender'],drop_first = True)
final

In [ ]:
from sklearn.preprocessing import LabelEncoder

label = LabelEncoder()

final['City'] = label.fit_transform(final['City'])

final['Education_Level'] = label.fit_transform(final['Education_Level'])

In [ ]:
final.head()

In [ ]:
box_plot=sns.boxplot(x='Total Business Value',data = final)
box_plot.set(xlabel='Total Business Value',ylabel='')

In [ ]:
Q1 = final['Total Business Value'].quantile(0.25)
Q3 = final['Total Business Value'].quantile(0.75)
IQR = Q3 - Q1

filter = (final['Total Business Value'] >= Q1 - 1.5 * IQR) & (final['Total Business Value']<= Q3 + 1.5 *IQR)
train1 = final.loc[filter]  
print("data loss percentage {}%".format(((len(final) - len(train1))/len(final))*100))

In [ ]:
box_plot=sns.boxplot(x='Salary',data=final)
box_plot.set(xlabel='Salary',ylabel='')

In [ ]:
Q1 = final['Salary'].quantile(0.25)
Q3 = final['Salary'].quantile(0.75)
IQR = Q3 - Q1

filter = (final['Salary'] >= Q1 - 1.5 * IQR) & (final['Salary']<= Q3 + 1.5 *IQR)
train2 = final.loc[filter]  
print("data loss percentage {}%".format(((len(final) - len(train2))/len(final))*100))

In [ ]:
final.to_csv('dataset/employee-attenuation-predicting.csv', index=False)

In [ ]:
train2.shape

In [ ]:
## Get the Fraud and the normal dataset 

not_stay= (train2['Target']== 0 )

stay= (train2['Target']== 1 )

In [ ]:

std_x = train2.loc[:,['Total Business Value','Age','Gender','City','Education_Level','Salary','Joining Designation','Designation','Quarterly Rating']]
std_x.shape

In [ ]:
y = train2.iloc[:,-1]
y.head()

In [ ]:
# Implementing Undersampling for Handling Imbalanced 
nm = NearMiss()
X_res,y_res=nm.fit_resample(std_x,y)

X_res.shape,y_res.shape

In [ ]:
print(sorted(list(final['Age'].unique())))

<a id="6"></a>
<h3 style='background:#92b0ac; border:0; color:black; padding-top:15px'><center>Training and Testing set</center></h1>

In [ ]:
#Split data into Train and test format
x_train,x_test,y_train,y_test = train_test_split(X_res,y_res,test_size = 0.25,random_state =105)

print('Shape of Training Xs:{}'.format(x_train.shape))
print('shape of Test Xt:{}'.format(x_test.shape))

<a id="7"></a>
<h1 style='background:#92b0ac; border:0; color:black; padding-top:15px'><center>MODEL BUILDING</center></h1>


In this section we will build and train several machine learning methods for our supervised task. The objective is to determine which model holds the most promise for further development (such as hyperparameter tuning).

We are comparing models using the mean absolute error. A baseline model that guessed the median value of the score was off by an average of 25 points.


In [ ]:
# Create an imputer object with a median filling strategy
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

# Train on the training features
imputer.fit(x_train)

# Transform both training data and testing data
X = imputer.transform(x_train)
X_test = imputer.transform(x_test)

print('Missing values in training features: ', np.sum(np.isnan(X)))
print('Missing values in testing features:  ', np.sum(np.isnan(X_test)))

# Make sure all values are finite
print(np.where(~np.isfinite(X)))
print(np.where(~np.isfinite(X_test)))


## 6.1. Scaling Features

The final step before we can start building our models is to scale the features. This is necessary because features are in different units, and we want to normalize the features so the units do not affect the algorithm. Eventhough, Linear Regression and Random Forest do not require feature scaling, but other methods, such as support vector machines (SVM) and k nearest neighbors (KNN), do require it because they take into account the Euclidean distance between observations.

There are two ways to scale features:
<ul>
<li>
For each value, subtract the mean of the feature and divide by the standard deviation of the feature. This is known as standardization and results in each feature having a mean of 0 and a standard deviation of 1.
</li>
<li>
For each value, subtract the minimum value of the feature and divide by the maximum minus the minimum for the feature (the range). This assures that all the values for a feature are between 0 and 1 and is called scaling to a range or normalization.
</li>
</ul>


In [ ]:
# Scaling Features

# Create the scaler object with a range of 0-1
scaler = MinMaxScaler(feature_range=(0, 1))

# Fit on the training data
scaler.fit(X)

# Transform both the training and testing data
X = scaler.transform(X)
X_test = scaler.transform(X_test)

# Convert y to one-dimensional array (vector)
y = np.array(y_train).reshape((-1, ))
y_test = np.array(y_test).reshape((-1, ))



### Models throught algorithms

We will compare five different machine learning models using the great Scikit-Learn library:

There are several ways for modeling:
<ul>
<li>
Logistic Regression
</li>
<li>
Support Vector Machine 
</li>
<li>
Random Forest Classifier
</li>
<li>
Gradient Boosting Classifier
    
</li>
</ul>


In [ ]:
# variables for saving results
accuracy_result = []
models_result = []

In [ ]:
# Function to calculate mean absolute error

def mae(y_true, y_pred):
    return np.mean(abs(y_true - y_pred))

# Takes in a model, trains the model, and evaluates the model on the test set
def fit_and_evaluate(model):
    
    # Train the model
    model.fit(X, y)
    
    # Make predictions and evalute
    model_pred = model.predict(X_test)
    model_mae = mae(y_test, model_pred)
    
    accuracy_result.append(model_mae*100)
    
    # Return the performance metric
    #return model_mae
    return {"model_mae": model_mae, "accuracy": model_mae*100, "model_pred": model_pred}

def fit_and_evaluate_model(model):
    
    # Train the model
    model.fit(X, y)
    
    # Make predictions and evalute
    model_pred = model.predict(X_test)
    model_mae = mae(y_test, model_pred)
    
    model_accuracy = accuracy_score(y_test, model_pred)
    model_accuracy = str(model_accuracy*100) + ' %'
    
    accuracy_result.append(model_accuracy)
    
    # Return the performance metric
    return {"mae": str(model_mae), "accuracy": model_accuracy, "model_pred": model_pred}

#### Logistic Regression Classifier

In [ ]:
models_result.append('Logistic Regression')

log_reg = LogisticRegression()
log_reg_model = fit_and_evaluate_model(log_reg)

In [ ]:
log_reg_model_pred=log_reg_model['model_pred']
print('Mean Absolute Error: ' + str(log_reg_model['mae']))
print('Accuracy: ' + str(log_reg_model['accuracy']))

In [ ]:
x_lg_res = X_res['Total Business Value'].values.reshape(-1,1)
y_lg_res = y_res

In [ ]:
lr_model = LogisticRegression()
lr_model.fit(x_lg_res,y_lg_res)
y_lr_pred = lr_model.predict(x_lg_res)

print(y_lr_pred)

In [ ]:
plt.scatter(x_lg_res,y_lg_res)
plt.xlabel('Total Business Value')
plt.ylabel('Target')
plt.plot(x_lg_res,y_lr_pred)

In [ ]:
theta0 = lr_model.intercept_
theta1 = lr_model.coef_
theta0, theta1

In [ ]:
y_lr_pred = lr_model.predict(np.array([80900]).reshape(1,1))
y_lr_pred

In [ ]:
y_lr_pred = lr_model.predict(np.array([49957]).reshape(1,1))
y_lr_pred

### Multivariant logistic regression

In [ ]:
attenuation_features = ['Education_Level','Total Business Value','Quarterly Rating']
x_lg_res = X_res[attenuation_features]
y_lg_res = y_res

In [ ]:
plt.subplot(2,2,1)
plt.scatter(X_res['Education_Level'],y_lg_res)
plt.subplot(2,2,2)
plt.scatter(X_res['Total Business Value'],y_lg_res)
plt.subplot(2,2,3)
plt.scatter(X_res['Quarterly Rating'],y_lg_res)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111,projection='3d')

x1_lg_res = X_res['Education_Level']
x2_lg_res = X_res['Total Business Value']

In [ ]:
ax.scatter(x1_lg_res,x2_lg_res,y_lg_res, c='r', marker = 'o')
ax.set_xlabel('Education Level')
ax.set_ylabel('Total Business Value')
ax.set_zlabel('Target')

In [ ]:
ax.scatter(x1_lg_res,x2_lg_res,y_lg_res, c='r', marker = 'o')

#### Support Vector Classification

In [ ]:
models_result.append('Support Vector Classification')

svm = SVC(C = 1000, gamma = 0.1)
svm_model = fit_and_evaluate_model(svm)

print('Mean Absolute Error: ' + str(svm_model['mae']))
print('Accuracy: ' + str(svm_model['accuracy']))

#### Decision Tree

In [ ]:
#Defining a function which will give us train and test accuracy for each classifier.
def train_test_error(y_train_data,y_test_data):
    train_error = ((y_train_data==y).sum())/len(y_train_data)*100
    test_error = ((y_test_data==y_test).sum())/len(y_test)*100
    accuracy_result.append(test_error)
    print('Decision Tree Performance: {}'.format(train_error) + " is the train accuracy")
    print('Decision Tree Performance: {}'.format(test_error) + " is the test accuracy")
    

In [ ]:
models_result.append('Decision Tree')

dtc = tree.DecisionTreeClassifier()
dtc_model = fit_and_evaluate_model(dtc)

print('Mean Absolute Error: ' + str(dtc_model['mae']))
print('Accuracy: ' + str(dtc_model['accuracy']))
#dtc.fit(X,y)
#train_predict = dtc.predict(X)
#test_predict = dtc.predict(X_test)


#### Random Forest Classifier

In [ ]:
models_result.append('Random Forest Classifier')

In [ ]:
rfc = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
rfc_metrics = fit_and_evaluate_model(rfc);
print('Mean Absolute Error: ' + str(rfc_metrics['mae']))
print('Accuracy: ' + str(rfc_metrics['accuracy']))

## All results from algorithms

In [ ]:
results = DataFrame({"Accuracy" : accuracy_result } , index = models_result)

results

### Random Forest - Hyperparameter tuning

In [ ]:
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rfc.get_params())

###### Random Hyperparameter Grid

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

###### Random Search Training

In [ ]:
# Use the random grid to search for best hyperparameters
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_rand = RandomForestClassifier();
rf_random = RandomizedSearchCV(estimator = rf_rand, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=0, n_jobs = -1)

# Fit the random search model
rf_random.fit(X, y)

In [ ]:
rf_random.best_params_

In [ ]:
def evaluate_rfc(model):
   # Train the model
    model.fit(X, y)
    
    # Make predictions and evalute
    model_pred = model.predict(X_test)
    model_mae = mae(y_test, model_pred)
    
    model_accuracy = accuracy_score(y_test, model_pred)
    model_accuracy = str(model_accuracy*100) + ' %'
    
    accuracy_result.append(model_accuracy)
    
    # Return the performance metric
    return {"mae": str(model_mae), "accuracy": model_accuracy, "model_pred": model_pred}

print(rfc_metrics)

In [ ]:
#best_random = rf_random.best_estimator_
#random_accuracy = evaluate_rfc(best_random)
#print(random_accuracy)
#Commented out: BECAUSE IT IS SLOWING DOWN 

###### Grid Search with Cross Validation

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)

In [ ]:
#grid_search.fit(X, y)
#grid_search.best_params_

#best_grid = grid_search.best_estimator_
#grid_accuracy = evaluate_rfc(best_grid)
#print(grid_accuracy) 
#Commented out: BECAUSE IT IS SLOWING DOWN

<a id="8"></a>
<h1 style='background:#92b0ac; border:0; color:black'><center>MODEL_REBUILDING(ANALYZING_AND_EVALUATION)</center></h1>

# 7. Rebuilding model ( analyzing and evaluation)

## Model Optimization
In machine learning, optimizing a model means finding the best set of hyperparameters for a particular problem.

### Hyperparameters

<ul>
<li>Model hyperparameters are best thought of as settings for a machine learning algorithm that are tuned by the data scientist before training. Examples would be the number of trees in the random forest, or the number of neighbors used in K Nearest Neighbors Regression.
</li>
<li>Model parameters are what the model learns during training, such as the weights in the linear regression.
</li>
</ul>

We as data scientists control a model by choosing the hyperparameters, and these choices can have a significant effect on the final performance of the model (although usually not as great of an effect as getting more data or engineering features).
<br/>
<i>Tuning the model hyperparameters</i> controls the balance of under vs over fitting in a model. We can try to correct for under-fitting by making a more complex model, such as using more trees in a random forest or more layers in a deep neural network. 

#### Hyperparameter using StandardScaler and PCA

In [ ]:
std_slc = StandardScaler()
pca = decomposition.PCA()
logistic_Reg = lr_model

In [ ]:
# Pipeline will helps us by passing modules one by one through GridSearchCV for which we want to get the best parameters.
pipe = Pipeline(steps=[('std_slc', std_slc),
                       ('pca', pca),
                       ('logistic_Reg', logistic_Reg)])

In [ ]:
n_components = list(range(1,X.shape[1]+1,1))
C = np.logspace(-4, 4, 50)
penalty = ['l2']

In [ ]:
parameters = dict(pca__n_components=n_components,
                  logistic_Reg__C=C,
                  logistic_Reg__penalty=penalty)

In [ ]:
clf_GS = GridSearchCV(pipe, parameters)
clf_GS.fit(X, y)

print('Best Penalty:', clf_GS.best_estimator_.get_params()['logistic_Reg__penalty'])
print('Best C:', clf_GS.best_estimator_.get_params()['logistic_Reg__C'])
print('Best Number Of Components:', clf_GS.best_estimator_.get_params()['pca__n_components'])
print(); 
print(clf_GS.best_estimator_.get_params()['logistic_Reg'])

#### Hyperparameter using GridSearchCV
Before using GridSearchCV, lets have a look on the important parameters.
<ul>
    <li>estimator: In this we have to pass the models or functions on which we want to use GridSearchCV
    </li>
    <li>param_grid: Dictionary or list of parameters of models or function in which GridSearchCV have to select the best.
    </li>
    </ul>


Scoring: It is used as a evaluating metric for the model performance to decide the best hyperparameters, if not especified then it uses estimator score.

## Learning Curves

In [ ]:
def plot_learning_curve(
    estimator,
    title,
    X,
    y,
    axes=None,
    ylim=None,
    cv=None,
    n_jobs=None,
    train_sizes=np.linspace(0.1, 1.0, 5),
):
    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = learning_curve(
        estimator,
        X,
        y,
        cv=cv,
        n_jobs=n_jobs,
        train_sizes=train_sizes,
        return_times=True,
    )
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(
        train_sizes,
        train_scores_mean - train_scores_std,
        train_scores_mean + train_scores_std,
        alpha=0.1,
        color="r",
    )
    axes[0].fill_between(
        train_sizes,
        test_scores_mean - test_scores_std,
        test_scores_mean + test_scores_std,
        alpha=0.1,
        color="g",
    )
    axes[0].plot(
        train_sizes, train_scores_mean, "o-", color="r", label="Training score"
    )
    axes[0].plot(
        train_sizes, test_scores_mean, "o-", color="g", label="Cross-validation score"
    )
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, "o-")
    axes[1].fill_between(
        train_sizes,
        fit_times_mean - fit_times_std,
        fit_times_mean + fit_times_std,
        alpha=0.1,
    )
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    fit_time_argsort = fit_times_mean.argsort()
    fit_time_sorted = fit_times_mean[fit_time_argsort]
    test_scores_mean_sorted = test_scores_mean[fit_time_argsort]
    test_scores_std_sorted = test_scores_std[fit_time_argsort]
    axes[2].grid()
    axes[2].plot(fit_time_sorted, test_scores_mean_sorted, "o-")
    axes[2].fill_between(
        fit_time_sorted,
        test_scores_mean_sorted - test_scores_std_sorted,
        test_scores_mean_sorted + test_scores_std_sorted,
        alpha=0.1,
    )
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    return plt


fig, axes = plt.subplots(3, 2, figsize=(10, 15))

X, y = load_digits(return_X_y=True)

title = "Learning Curves (Naive Bayes)"
# Cross validation with 50 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=50, test_size=0.2, random_state=0)

estimator = GaussianNB()
plot_learning_curve(
    estimator, title, X, y, axes=axes[:, 0], ylim=(0.7, 1.01), cv=cv, n_jobs=4
)

title = r"Learning Curves (SVM, RBF kernel, $\gamma=0.001$)"
# SVC is more expensive so we do a lower number of CV iterations:
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
estimator = svm# svm type of SVC(C = 1000, gamma = 0.1)
plot_learning_curve(
    estimator, title, X, y, axes=axes[:, 1], ylim=(0.7, 1.01), cv=cv, n_jobs=4
)

plt.show()

<a id="9"></a>
<h1 style='background:#92b0ac; border:0; color:black'><center>CONFUSION_MATRIX</center></h1>


# 8. Confusion Matirx

### Compute confusion matrix

In [ ]:
cnf_matrix = confusion_matrix(y_test, log_reg_model_pred)

np.set_printoptions(precision=2)
cnf_matrix

print(classification_report(y_test,log_reg_model_pred))

<a id="10"></a>
<h1 style='background:#92b0ac; border:0; color:black'><center>CONCLUSION</center></h1>


# 9. Conclusion

### Logisitc Regression

Hypothesis: h<sub>Θ</sub>(x)=g(Θ<sup>T</sup>*x) , h<sub>Θ</sub>(x)=g(Θ<sub>0</sub>*x<sub>0</sub>+ Θ<sub>1</sub>*x<sub>1</sub>+ ... + Θ<sub>n</sub>*x<sub>n</sub>) , g(z)= 1/(1- e<sup>-z</sup>)
<br/> 
=> h<sub>Θ</sub>(x)=1/(1- e<sup>Θ<sub>0</sub>*x<sub>0</sub>+ Θ<sub>1</sub>*x<sub>1</sub>+ ... + Θ<sub>n</sub>*x<sub>n</sub></sup>)

Suppose predict “y=1" for h<sub>Θ</sub>(x)>=1/2 or "y=0" otherwise

Logisitc regression cost function: J(Θ) = 1/(m) Σ {Cost (h<sub>Θ</sub>(x), y<sup>i</sup>)}=-1/m Σ{y<sup>i</sup>*log(h<sub>Θ</sub>(x))+(1-y<sup>i</sup>)log(1- h<sub>Θ</sub>(x))}